In [1]:
import numpy as np
from openfermion.transforms import get_sparse_operator
from openfermion.utils import pauli_exp_to_qasm
from openfermion.ops import QubitOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from pyscf import ci
from pyscf import mp
from openfermionpyscf import run_pyscf
from openfermion.hamiltonians import MolecularData
import os

from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize

from openfermion.config import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize


In [2]:
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.7414]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [3]:
orbitals = h2_molecule.n_orbitals
qubit= h2_molecule.n_qubits
elektron=h2_molecule.n_electrons

In [4]:
from openfermion.utils import uccsd_singlet_generator, uccsd_generator
initial_amplitudes = np.linspace(0,0.05677,2)
uccsd_fermion=uccsd_singlet_generator(initial_amplitudes,qubit,elektron)
#b=uccsd_generator(initial_amplitudes,qubit,elektron)
print(uccsd_fermion)

-0.05677 [0^ 2 1^ 3] +
-0.05677 [1^ 3 0^ 2] +
0.05677 [2^ 0 3^ 1] +
0.05677 [3^ 1 2^ 0]


In [5]:
uccsd_jw=jordan_wigner(uccsd_fermion)
print(uccsd_jw)

-0.0141925j [X0 X1 X2 Y3] +
-0.0141925j [X0 X1 Y2 X3] +
0.0141925j [X0 Y1 X2 X3] +
-0.0141925j [X0 Y1 Y2 Y3] +
0.0141925j [Y0 X1 X2 X3] +
-0.0141925j [Y0 X1 Y2 Y3] +
0.0141925j [Y0 Y1 X2 Y3] +
0.0141925j [Y0 Y1 Y2 X3]


In [6]:
#membuat ansatz

import openfermion as of
import cirq
import openfermioncirq as ofc
import numpy as np
class MyAnsatz(ofc.VariationalAnsatz):
    def params(self):      
        return [cirq.Symbol('theta_0'),cirq.Symbol('theta_1')]
    def operations(self, qubits):    
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0),cirq.H(q1),cirq.H(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0),cirq.H(q1),cirq.H(q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q3)
        
        yield cirq.H(q0),cirq.H(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0),cirq.H(q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        yield cirq.H(q3)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2),cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q1)
        yield cirq.H(q2),cirq.H(q3)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1),cirq.H(q2),cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q1),cirq.H(q2),cirq.H(q3)
        
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q1)
        yield cirq.H(q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        yield cirq.H(q3)
        
        
        
        
        
        
    def _generate_qubits(set):
        return cirq.LineQubit.range(4)

In [7]:
ansatz=MyAnsatz()

In [9]:
import openfermionpyscf as ofpyscf
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)
hamiltonian_jw=jordan_wigner(hamiltonian)

In [14]:

objective = ofc.HamiltonianObjective(hamiltonian_jw)


q0, q1, _, _ = ansatz.qubits
reference_circuit = cirq.Circuit.from_ops(cirq.X(q0), cirq.X(q1))


study = ofc.VariationalStudy(
    name='my_hydrogen_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=reference_circuit)

print(study.num_params)
print(ansatz.default_initial_params())

2
[0. 0.]


In [21]:
a = np.linspace(-0.01, 0.01)
b = np.linspace(-0.008, -0.009)
xg,yg=np.meshgrid(x,y)
hasil=[study.value_of([x,y]) for x, y in zip(a, b)]
print(hasil)
print(a)

[-1.1119007995939767, -1.1129281012426817, -1.113933709791527, -1.1149175969825313, -1.115879735177788, -1.1168200973602427, -1.1177386571344854, -1.118635388727489, -1.1195102669893622, -1.120363267394062, -1.1211943660400994, -1.1220035396512225, -1.122790765577086, -1.1235560217938951, -1.1242992869050417, -1.125020540141712, -1.1257197613634806, -1.1263969310588935, -1.1270520303460188, -1.1276850409729877, -1.1282959453185197, -1.1288847263924235, -1.1294513678360811, -1.1299958539229191, -1.1305181695588553, -1.131018300282727, -1.1314962322667077, -1.131951952316698, -1.132385447872701, -1.1327967070091853, -1.133185718435416, -1.1335524714957859, -1.133896956170105, -1.1342191630739005, -1.134519083458664, -1.134796709212122, -1.1350520328584417, -1.1352850475584577, -1.135495747109862, -1.1356841259473713, -1.135850179142895, -1.1359939024056545, -1.1361152920823234, -1.1362143451571132, -1.1362910592518607, -1.1363454326260922, -1.1363774641770674, -1.1363871534398091, -1.136

In [13]:
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'nelder-mead'},
    options={'maxiter': 1000})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Nilai optimum: -1.1363882043260851
parameter optimasinya:
[ 0.0097644  -0.00834586]


In [11]:
print(molecule.ccsd_energy)

-1.1372703406409188


In [14]:
jw_hamiltonian=jordan_wigner(hamiltonian)
print(jw_hamiltonian)




-0.0988639693354571 [] +
-0.045322202052874 [X0 X1 Y2 Y3] +
0.045322202052874 [X0 Y1 Y2 X3] +
0.045322202052874 [Y0 X1 X2 Y3] +
-0.045322202052874 [Y0 Y1 X2 X3] +
0.17119774903432944 [Z0] +
0.16862219158920938 [Z0 Z1] +
0.120544822053018 [Z0 Z2] +
0.165867024105892 [Z0 Z3] +
0.17119774903432947 [Z1] +
0.165867024105892 [Z1 Z2] +
0.120544822053018 [Z1 Z3] +
-0.22278593040418496 [Z2] +
0.17434844185575687 [Z2 Z3] +
-0.22278593040418496 [Z3]


In [31]:
from openfermion.utils import error_bound, error_operator
a=QubitOperator('')
b=QubitOperator('X0 X1 Y2 Y3',-0.0988639693354571)
c=QubitOperator('X0 Y1 Y2 X3',-0.045322202052874)
d=QubitOperator('Y0 X1 X2 Y3',0.045322202052874)
e=QubitOperator('Y0 Y1 X2 X3',0.045322202052874)
f=QubitOperator('Z0',0.17119774903432944)
g=QubitOperator('Z0 Z1',0.16862219158920938)
h=QubitOperator('Z0 Z2',0.120544822053018)
i=QubitOperator('Z1',0.17119774903432947)
j=QubitOperator('Z1 Z2',0.165867024105892)
k=QubitOperator('Z1 Z3',0.120544822053018)
l=QubitOperator('Z2',-0.22278593040418496)
m=QubitOperator('Z2 Z3',0.17434844185575687)
n=QubitOperator('Z3',-0.22278593040418496)

terms=[a,b,c,d,e,f,g,h,i,k,l,m,n]
er=error_bound(terms,tight=False)
print(er)

0.583218227476381


In [33]:
err=error_operator(terms,series_order=2)
print(err)

(0.0011928794631486436+0j) [X0 X1 X2 X3] +
(-0.0026015144002038493+0j) [X0 X1 Y2 Y3] +
(0.00037364593144043344+0j) [X0 Y1 X2 Y3] +
(-0.0005119096890636866+0j) [X0 Y1 Y2 X3] +
(0.0018733144139546731+0j) [Y0 X1 X2 Y3] +
(0.0009877587934505533+0j) [Y0 X1 Y2 X3] +
(0.00119261205150918+0j) [Y0 Y1 X2 X3] +
(0.0002160228855460254+0j) [Y0 Y1 Y2 Y3] +
(-0.0018800042226705014+0j) [Z0] +
(-0.00022474682834517875+0j) [Z0 Z1 Z2] +
(-0.001716412170817677+0j) [Z0 Z1 Z3] +
(-0.00022474682834517875+0j) [Z0 Z2 Z3] +
(6.115477649235432e-05+0j) [Z1] +
(0.0017164121708176773+0j) [Z1 Z2 Z3] +
(0.0019293005276117426+0j) [Z2] +
(0.0004376351851392439+0j) [Z3]


In [34]:
from openfermion.ops import QubitOperator
from openfermion.utils import pauli_exp_to_qasm
for line in pauli_exp_to_qasm([QubitOperator('X0 X1 X2 Y3',-0.0141925),QubitOperator('X0 X1 Y2 X3',-0.0141925), QubitOperator('X0 Y1 X2 X3', 0.0141925), 
                              QubitOperator('X0 Y1 Y2 Y3',-0.0141925j),QubitOperator('Y0 X1 X2 X3', 0.0141925), QubitOperator('Y0 X1 Y2 Y3',-0.0141925),
                              QubitOperator('Y0 Y1 X2 Y3',0.0141925), QubitOperator('Y0 Y1 Y2 X3',0.0141925)]):
    print(line)


H 0
H 1
H 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
H 2
Rx -1.5707963267948966 3
H 0
H 1
Rx 1.5707963267948966 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 2
H 3
H 0
Rx 1.5707963267948966 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
H 2
H 3
H 0
Rx 1.5707963267948966 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.5707963267948966 0
H 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
H 2
H 3
Rx 1.5707963267948966 0
H 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
Rx -1.5707963267948966 2
Rx -1.57079